Transfer learning

Transfer Learning means:

Using a pre-trained model that already learned features from millions of images, and adapting it to your own task (cats vs dogs).

Instead of training from scratch, you reuse knowledge.

Common Pre-Trained Models

MobileNetV2	,
ResNet50,	
VGG16,
InceptionV3	,

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D ,Dense




c:\Users\bhair\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(
    rescale = 1./25
)

In [3]:
train_data = train_datagen.flow_from_directory(
    "dataset/train",
    target_size = (224,224),
    batch_size = 32,
    class_mode = "binary"
)

test_data = test_datagen.flow_from_directory(
    "dataset/test",
    target_size = (224,224),
    batch_size = 32,
    class_mode = "binary"
)
#Why 224×224?
#Pre-trained models expect this input size.

Found 2 images belonging to 2 classes.
Found 2 images belonging to 2 classes.


 #imagenet → trained on 1.4M images
                            #include_top=False → remove original classifier

In [4]:
base_model = MobileNetV2(
    weights ="imagenet",
    include_top= False,
    input_shape = (224, 224, 3)
)
base_model.trainable = False

Freeze the base model

In [5]:

#base_model.trainable = False locks the pre-trained weights so only 
# your new layers learn, preventing overfitting and speeding up training.

GlobalAveragePooling2D() takes the whole image features and compresses them into a small summary so the model can decide easily without overthinking.

In [6]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(), 
    Dense(64, activation="relu"),
    Dense(1, activation="sigmoid")
])

In [7]:
model.compile(
    optimizer = "adam",
    loss = "binary_crossentropy",
    metrics = ["accuracy"]
)

In [8]:
model.fit(
    train_data,
    epochs = 10,
    validation_data =test_data
)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.5000 - loss: 0.7360 - val_accuracy: 0.5000 - val_loss: 0.4972
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 1.0000 - loss: 0.2460 - val_accuracy: 0.5000 - val_loss: 0.5578
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 1.0000 - loss: 0.1295 - val_accuracy: 0.5000 - val_loss: 0.5341
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 1.0000 - loss: 0.0573 - val_accuracy: 0.5000 - val_loss: 0.4889
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - accuracy: 1.0000 - loss: 0.0299 - val_accuracy: 0.5000 - val_loss: 0.4418
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - accuracy: 1.0000 - loss: 0.0234 - val_accuracy: 1.0000 - val_loss: 0.3891
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 1.0000 - loss: 0.0084 - val_accuracy: 1.0000 - val_loss: 0.3572
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - accuracy: 1.0000 - loss: 0.0046 - val_accuracy: 1.0000 - val_loss: 0.

In [10]:
loss, acc =model.evaluate(test_data)
print(acc)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 1.0000 - loss: 0.2659
1.0


Fine Tuning


Unfreezing some top layers of the pre-trained model and training them slightly on your data.

In [ ]:
for layers in base_model.layers[:-30]:
    layers.trainable = False
#Only the last 30 layers adapt to your dataset.